<a href="https://colab.research.google.com/github/RihanNasar/ririai/blob/main/riri-backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y transformers audiocraft

# Install correct versions of dependencies
!pip install fastapi uvicorn pyngrok nest_asyncio
!pip install torch==2.1.0 torchaudio
!pip install numpy==1.25.2
!pip install transformers==4.30.2  # Use an older, stable version

# Increase Python's recursion limit to handle deep imports
import sys
sys.setrecursionlimit(3000)  # Default is usually 1000

# Now install audiocraft from source
!pip install git+https://github.com/facebookresearch/audiocraft.git

Found existing installation: transformers 4.51.2
Uninstalling transformers-4.51.2:
  Successfully uninstalled transformers-4.51.2
Found existing installation: audiocraft 1.4.0a2
Uninstalling audiocraft-1.4.0a2:
  Successfully uninstalled audiocraft-1.4.0a2
  Using cached transformers-4.30.2-py3-none-any.whl.metadata (113 kB)
  Using cached tokenizers-0.13.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
Using cached tokenizers-0.13.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4

In [ ]:
!ngrok authtoken 2tP43jwLuDty2X5OypLN9hkWTgD_sbghpnebfDGb4NdNAWZp

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!killall ngrok

ngrok: no process found


In [ ]:

# Import all necessary libraries
import numpy as np
import os
import sys
import threading
import time
import requests
from datetime import datetime
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import torch
import torchaudio
from IPython.display import display, Javascript, HTML
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse
from fastapi.staticfiles import StaticFiles
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write

# Fix deprecated numpy bool alias if needed
if not hasattr(np, 'bool'):
    np.bool = bool

# --- FastAPI Setup ---
app = FastAPI()

# --- CORS Middleware ---
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# --- Ensure static directory exists ---
static_dir = "/content/static"
os.makedirs(static_dir, exist_ok=True)

# --- Mount static files ---
app.mount("/static", StaticFiles(directory=static_dir), name="static")

# --- Increase recursion limit for transformers ---
sys.setrecursionlimit(3000)

# --- Define public_url as global ---
public_url = None

# --- Load MusicGen model ---
print("📦 Loading MusicGen model...")
sys.stdout.flush()
model = MusicGen.get_pretrained('facebook/musicgen-small', device="cpu")
model.set_generation_params(duration=10)
print("✅ MusicGen model loaded.")
sys.stdout.flush()

# --- Ngrok Setup ---
print("🔗 Setting up ngrok tunnel...")
public_url = ngrok.connect(8000)
print("🌐 Public URL:", public_url)

# --- Routes ---

@app.get("/")
async def root():
    return {"message": "Welcome to the MusicGen API 🎶", "status": "online"}

@app.get("/generate")
async def generate_music(prompt: str):
    print(f"🎵 Received prompt: {prompt}")
    descriptions = [prompt]

    try:
        wav = model.generate(descriptions)
    except Exception as e:
        print("❌ Error during music generation:", e)
        return {"error": f"Music generation failed: {str(e)}"}

    # Use a hash of the prompt to create a unique filename
    output_name = f"music_{abs(hash(prompt)) % 10000}"
    output_path = os.path.join(static_dir, output_name)
    final_file_path = output_path + ".wav"

    try:
        audio_write(output_path, wav[0].cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)
        print("✅ Audio written:", final_file_path)
    except Exception as e:
        print("❌ Error writing audio:", e)
        return {"error": f"Audio writing failed: {str(e)}"}

    if os.path.exists(final_file_path):
        return FileResponse(
            path=final_file_path,
            media_type="audio/wav",
            filename=f"{output_name}.wav"
        )

    return {"error": "Music file not found"}

# --- Test route to return a static test audio file ---
@app.get("/test-audio")
async def test_audio():
    path = os.path.join(static_dir, "test.wav")
    if os.path.exists(path):
        return FileResponse(path=path, media_type="audio/wav", filename="test.wav")
    return {"error": "Test audio not found"}

# --- Keep Colab and ngrok alive ---
def keep_alive():
    global public_url  # Declare global at the beginning of the function
    last_ngrok_restart = datetime.now()

    while True:
        current_time = datetime.now()

        # Display a message and interact with the page to keep Colab alive
        display(HTML(f"<p style='color:green'>Keeping session alive... MusicGen API is running at: {public_url}</p>"))
        display(Javascript('''
        function ClickConnect(){
            console.log("Keeping Colab alive...");
            document.querySelector("colab-toolbar-button#connect").click();
        }
        setInterval(ClickConnect, 60000);
        '''))

        # Check if ngrok is still working by making a request to the root endpoint
        try:
            response = requests.get(f"{public_url}")
            if response.status_code != 200:
                raise Exception("ngrok tunnel not responding properly")
        except Exception as e:
            print(f"ngrok tunnel issue detected: {e}")

            # If ngrok has been running for more than 1.5 hours, restart it
            # (Free ngrok tunnels expire after ~2 hours)
            time_diff = current_time - last_ngrok_restart
            if time_diff.total_seconds() > 5400:  # 1.5 hours in seconds
                print("Restarting ngrok tunnel...")

                # Kill existing ngrok processes
                ngrok.kill()

                # Create a new tunnel
                public_url = ngrok.connect(8000)
                last_ngrok_restart = current_time

                print(f"New ngrok URL: {public_url}")

        # Sleep for 5 minutes before checking again
        time.sleep(300)

# --- Start the server and keep-alive thread ---
def start_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Apply nest_asyncio to allow running asyncio in notebooks
nest_asyncio.apply()

# Start the server in a separate thread
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

# Start the keep-alive thread
keep_alive_thread = threading.Thread(target=keep_alive, daemon=True)
keep_alive_thread.start()

print("✅ MusicGen API is now running!")
print("🔗 API Endpoint:", f"{public_url}/generate?prompt=your_music_prompt_here")
print("🔄 Anti-disconnect measures are active for both Colab and ngrok")

📦 Loading MusicGen model...


/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


✅ MusicGen model loaded.
🔗 Setting up ngrok tunnel...
🌐 Public URL: NgrokTunnel: "https://f4b3-34-90-103-229.ngrok-free.app" -> "http://localhost:8000"
✅ MusicGen API is now running!
🔗 API Endpoint: NgrokTunnel: "https://f4b3-34-90-103-229.ngrok-free.app" -> "http://localhost:8000"/generate?prompt=your_music_prompt_here
🔄 Anti-disconnect measures are active for both Colab and ngrok


INFO:     Started server process [2247]


<IPython.core.display.Javascript object>

INFO:     Waiting for application startup.
INFO:     Application startup complete.
